In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.contrib import rnn
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import matplotlib.pylab as plt
import time

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
#Read the training file path
file_path = 'Processed_Data.csv'
raw_data = pd.read_csv(file_path)

In [3]:
def parse_fl(data):
    # This function mainly to transfer the raw csv file to 
    # training features and labels. 
    
    # Select the features in the input data
    features = data[:, 0:122]
    # Select the labels in the input data
    labels = data[:, 122]
    #trans the label from text to

    return features, labels

In [4]:
train_data = raw_data.values

In [5]:
# Prepare the training features and labels 
train_features, train_labels = parse_fl(train_data)

In [6]:
temp_labels = []

for i in range(len(train_labels)):
    if (train_labels[i] == 1):
        temp_labels.append([1, 0])
    else:
        temp_labels.append([0, 1])

train_labels = temp_labels
train_labels = np.array(train_labels)

In [9]:
learning_rate = 0.01
n_classes = 2
display_step = 10
training_cycles = 100
hidden_units = 20
input_features = 122
time_steps = 1

In [11]:
newtrain_X = train_features.reshape(len(train_features), time_steps, -1)
newtrain_Y = train_labels.reshape(len(train_labels), n_classes)

(125973, 122)
(125973, 1, 122)
(125973, 2)


In [13]:
x = tf.placeholder(tf.float32, [None, time_steps, input_features], name='x-input')
y = tf.placeholder(tf.float32, [None, n_classes], name='y-input')

(?, 1, 122)
(?, 2)


In [14]:
weight = tf.Variable(tf.random_normal([hidden_units, n_classes]), name='weight')
bias = tf.Variable(tf.random_normal(shape=[n_classes]), name='bias')
print(weight.shape)
print(bias.shape)

(20, 2)
(2,)


In [15]:
cell = tf.contrib.rnn.BasicRNNCell(hidden_units)
init_state = cell.zero_state(len(newtrain_X), dtype=tf.float32)
outputs, final_state = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)
outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
results = tf.matmul(outputs[-1], weight) + bias

Tensor("unstack:0", shape=(?, 20), dtype=float32)
(20, 2)
(?, 2)
Tensor("add:0", shape=(?, 2), dtype=float32)


In [16]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=results, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

(?, 2)


In [17]:
correct_pred = tf.equal(tf.argmax(results, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [18]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [19]:
for i in range (training_cycles):
    _,c = sess.run([optimizer,cost], feed_dict = {x:newtrain_X, y:newtrain_Y})
    
    if (i) % display_step == 0:
        print ("Cost for the training cycle : ",i," : is : ",sess.run(cost, feed_dict ={x :newtrain_X,y:newtrain_Y}))

Cost for the training cycle :  0  : is :  0.61651224
Cost for the training cycle :  10  : is :  0.13260826
Cost for the training cycle :  20  : is :  0.1134961
Cost for the training cycle :  30  : is :  0.101827495
Cost for the training cycle :  40  : is :  0.09326477
Cost for the training cycle :  50  : is :  0.0881147
Cost for the training cycle :  60  : is :  0.08493735
Cost for the training cycle :  70  : is :  0.0823467
Cost for the training cycle :  80  : is :  0.08014731
Cost for the training cycle :  90  : is :  0.07828935


In [21]:
correct = tf.equal(tf.argmax(outputs, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
print('Accuracy on the overall test set is :',accuracy.eval({x:newtrain_X, y:newtrain_Y}))

NameError: name 'newtest_X' is not defined

In [ ]:
# Calculating Accuracy through another procedure
n = tf.cast(labels_class,tf.int64)
newaccuracy = tf.contrib.metrics.accuracy(pred_class,n)
print ("accuracy calcualted using tf.contrib", sess.run (newaccuracy, feed_dict = {x:newtest_X,y:newtest_Y}))

#Calculations performed manually for other metrics
TP = conf[0,0]
FN = conf [0,1]
FP = conf[1,0]
TN = conf[1,1]

AccConf = (TP+TN)/(TP+FP+TN+FN)
print ("Accuracy calculated manually through confusion matrix", sess.run (AccConf, feed_dict = {x:newtest_X,y:newtest_Y}))

# Precision
Precision = TP/(TP+FP)
print ("Precision \n",sess.run(Precision,feed_dict ={x:newtest_X,y:newtest_Y}))

#Recall also known as Sensitivity
Recall = TP/(TP+FN)
print ("Recall (DR) - Sensitivity [True Positive Rate]\n", sess.run(Recall,feed_dict={x:newtest_X,y:newtest_Y}))

# Specificity

Specificity = TN/(TN+FP)
print ("Specificity \n", sess.run(Specificity,feed_dict={x:newtest_X,y:newtest_Y}))

#F score
FScore = 2*((Precision*Recall)/(Precision+Recall))
print ("F1 Score is \n",sess.run(FScore,{x:newtest_X,y:newtest_Y}))

#False Alarm Rate
FAR = FP/(FP+TN)
print ("False Alarm Rate also known as False Positive Rate \n",sess.run(FAR,feed_dict ={x:newtest_X,y:newtest_Y}))

#Efficiency
Efficiency = Recall/FAR
print("Efficincy is \n",sess.run(Efficiency,feed_dict = {x:newtest_X,y:newtest_Y}))